In [1]:
import ctypes
import numpy as np
import math
import time

#Define point structure
class HeliosPoint(ctypes.Structure):
    #_pack_=1
    _fields_ = [('x', ctypes.c_uint16),
                ('y', ctypes.c_uint16),
                ('r', ctypes.c_uint8),
                ('g', ctypes.c_uint8),
                ('b', ctypes.c_uint8),
                ('i', ctypes.c_uint8)]
class Dac:
    def __init__(self):
        #Load and initialize library
        self.HeliosLib = ctypes.cdll.LoadLibrary("./libHeliosDacAPI.so")
        self.num_devices = self.HeliosLib.OpenDevices()
        print("Found ", self.num_devices, "Helios DACs")
        # Define limits
        self.xy_max = int(2**12-1)

dac = Dac()




Found  1 Helios DACs


In [171]:
# Define update function
def dacpos(x, y, r, g, b, lum, dac):
    #Constants
    dac_rate = 65000 # very low, change to higher
    dac_idx = 0
    num_points = 1
    
    # Maps x, y from [-1, 1] to [0, xy_max]
    x_dac = int(float((x + 1)/2)*dac.xy_max)
    y_dac = int(float((y + 1)/2)*dac.xy_max)
    
    # Scale the colors by the luminance
    r_dac = int(lum*r)
    g_dac = int(lum*g)
    b_dac = int(lum*b)
    # Make frame
    frame = HeliosPoint(x_dac, y_dac, r_dac, g_dac, b_dac, 255)
    # Post frame
    status_attempts = 0
    while (status_attempts < 512 and dac.HeliosLib.GetStatus(dac_idx) != 1):
        status_attempts += 1
        print('Too fast!')
    dac.HeliosLib.WriteFrame(dac_idx, dac_rate, 0, ctypes.pointer(frame), num_points)

    
def color_correction(arr_color):
    '''
    Corrects the nonlinearities in the color curve 
    '''
    arr_color = np.exp(np.log(2)*np.power(arr_color,2)) - 1
    # Scale to the lower cutoff
    lower_cutoff = 0.18
    arr_color = arr_color*(1.0-lower_cutoff) + lower_cutoff
    
    
    return arr_color

def position_corection(arr_pos):
    '''
    Corrects x-y directions so that (x=-1,y=-1) is bottom left
    Scales the entire pattern so that it does not clip (amps are weird)
    '''
    # Reverse direction of x and y
    arr_pos *= -1
    
    max_scale = 0.75
    return arr_pos*max_scale
    
    
    
def dacpos_batch(arr_pos, arr_color, dac):
    '''
    1) Scales and color corrects 
    2) Converts from unit space to DAC coordinates
    3) Produces a DAC compatible frame and displays it
    '''
    # Scale the pattern 
    arr_pos = position_corection(arr_pos)
    # Format the position array (0-1.0) -> int(0-4095)
    arr_pos = (((arr_pos+1)/2.0)*dac.xy_max).astype(np.int32)
    
    # Perform color correction
    arr_color = color_correction(arr_color)
    # Format the color array (0-1.0) -> int(0-255)
    arr_color = (arr_color*255).astype(np.int32)
    # Fill a heliospoint arr with these values
    num_points = len(arr_pos)
    print('making a frame with ', num_points)
    
    frameType = HeliosPoint * num_points
    frame = frameType()
    # Fill the frame
    for idx in range(num_points):
        frame[idx] =     HeliosPoint(int(arr_pos[idx, 0]), 
                                     int(arr_pos[idx, 1]), 
                                     int(arr_color[idx, 0]), 
                                     int(arr_color[idx, 1]),
                                     int(arr_color[idx, 2]),
                                     int(255))
    # Write the values out 
    statusAttempts = 0
    # Make 512 attempts for DAC status to be ready. After that, just give up and try to write the frame anyway
    while (statusAttempts < 512 and dac.HeliosLib.GetStatus(0) != 1):
        statusAttempts += 1
    dac.HeliosLib.WriteFrame(0, 350, 0, ctypes.pointer(frame), num_points)
    
    

    

In [172]:
T = 1000
theta = np.linspace(0, 2*np.pi, T)
pattern = np.zeros((T, 5))
pattern[:, 0] = np.cos(theta)
pattern[:, 1] = np.sin(theta)
pattern[:, 2:5] = np.concatenate([np.linspace(0, 1, T//2)[:,np.newaxis], np.linspace(1, 0, T//2)[:,np.newaxis]])
#pattern[:, 4] = 0

while(True):
    dacpos_batch(pattern[:, 0:2], pattern[:,2:5], dac)
    print('hello')
    break


making a frame with  1000
hello


In [173]:
T = 1000
pattern = np.zeros((T, 5))
pattern[:, 0:2] = np.concatenate([np.linspace(-1, 1, T//2)[:,np.newaxis], np.linspace(1, -1, T//2)[:,np.newaxis]])
pattern[:, 2:5] = np.concatenate([np.linspace(0, 1, T//2)[:,np.newaxis], np.linspace(1, 0, T//2)[:,np.newaxis]])

while(True):
    dacpos_batch(pattern[:, 0:2], pattern[:,2:5], dac)
    print('hello')
    break

making a frame with  1000
hello


In [174]:
import cv2
def draw_bitmap(arr_pos, arr_color, dims=(400,400)):
    '''
    Draws a pattern as if we were drawing it on a wall
    '''
    # Copy the arrays so we don't change them 
    arr_pos = arr_pos.copy()
    arr_color = arr_color.copy()
    # Map the positions
    arr_pos[:,0] *= -1
    arr_pos = (((arr_pos+1)/2)*dims).astype(np.int)
    # Map the colors
    arr_color = arr_color[:,::-1]
    T = len(arr_pos)
    img = np.zeros(dims+(3,))
    # Draw points
    for t in range(1, T):
        cv2.line(img, arr_pos[t-1], arr_pos[t], color=arr_color[t-1], thickness=2)
    cv2.imshow('img', img)
    cv2.waitKey(0)
    
    

In [170]:
draw_bitmap(pattern[:,0:2], pattern[:,2:5])


In [163]:
pattern[:, 2:5]

array([[0.        , 0.        , 0.        ],
       [0.00200401, 0.00200401, 0.00200401],
       [0.00400802, 0.00400802, 0.00400802],
       ...,
       [0.00400802, 0.00400802, 0.00400802],
       [0.00200401, 0.00200401, 0.00200401],
       [0.        , 0.        , 0.        ]])